In [1]:
# The code was removed by Watson Studio for sharing.

,CloseTime,OpenPrice,ClosePrice,HighPrice,LowPrice,Volume
0,2013-04-01 23:58:00,103.83018,103.99900,103.99900,103.83018,2.200000
1,2013-04-01 23:57:00,103.49951,103.49951,103.49951,103.49951,3.148377
2,2013-04-01 23:55:00,103.00007,103.00007,103.00007,103.00007,5.400000
3,2013-04-01 23:40:00,103.22979,102.53000,103.22979,102.53000,5.816000
4,2013-04-01 23:39:00,102.99999,103.70250,103.70250,102.99999,50.000000


In [2]:
bit_data["date"]=pd.to_datetime(bit_data["CloseTime"]).dt.date
group=bit_data.groupby("date")
data=group["ClosePrice"].mean()

In [6]:
close_train=data.iloc[:len(data)-50]
close_test=data.iloc[len(close_train):]

#feature scalling (set values between 0-1)
close_train=np.array(close_train)
close_train=close_train.reshape(close_train.shape[0],1)
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(0,1))
close_scaled=scaler.fit_transform(close_train)

timestep=50
x_train=[]
y_train=[]

for i in range(timestep,close_scaled.shape[0]):
    x_train.append(close_scaled[i-timestep:i,0])
    y_train.append(close_scaled[i,0])

x_train,y_train=np.array(x_train),np.array(y_train)
x_train=x_train.reshape(x_train.shape[0],x_train.shape[1],1) #reshaped for RNN
print("x_train shape= ",x_train.shape)
print("y_train shape= ",y_train.shape)

x_train shape=  (2466, 50, 1)
y_train shape=  (2466,)


In [ ]:
from sklearn.metrics import mean_absolute_error
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout,Flatten

model=Sequential()

model.add(LSTM(10,input_shape=(None,1),activation="relu"))

model.add(Dense(1))

model.compile(loss="mean_squared_error",optimizer="nadam")

model.fit(x_train,y_train,epochs=100,batch_size=32)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
 320/2466 [==>...........................] - ETA: 47:47 - loss: 0.1100

In [ ]:
inputs=data[len(data)-len(close_test)-timestep:]
inputs=inputs.values.reshape(-1,1)
inputs=scaler.transform(inputs)
x_test=[]
for i in range(timestep,inputs.shape[0]):
    x_test.append(inputs[i-timestep:i,0])
x_test=np.array(x_test)
x_test=x_test.reshape(x_test.shape[0],x_test.shape[1],1)

In [ ]:
predicted_data=model.predict(x_test)
predicted_data=scaler.inverse_transform(predicted_data)

In [ ]:
data_test=np.array(close_test)
data_test=data_test.reshape(len(data_test),1)
plt.figure(figsize=(8,4), dpi=80, facecolor='w', edgecolor='k')
plt.plot(data_test,color="r",label="true result")
plt.plot(predicted_data,color="b",label="predicted result")
plt.legend()
plt.xlabel("Time(50 days)")
plt.ylabel("Close Values")
plt.grid(True)
plt.show()
import sklearn.datasets as datasets
import matplotlib.pyplot as plt
X, Y = datasets.make_blobs(centers=4, cluster_std=0.5, random_state=0)
plt.scatter(X[:,0], X[:,1]);